In [ ]:
import sys

sys.path.append("../")

from src.data.query_snowflake import (
    read_DALI_meta,
    make_DALI_week_extremes,
    write_DALI_week_extremes,
    read_DALI_week_extremes,
)

In [ ]:
%%time
df_meta = read_DALI_meta()
df_meta

In [ ]:
%%time
df_extremes = make_DALI_week_extremes(boxid=df_meta["boxid"])
df_extremes

In [ ]:
df_extremes.info()

In [ ]:
%%time
write_DALI_week_extremes(df_extremes)

In [ ]:
%%time
df_extremes_new = read_DALI_week_extremes(L="sumli")

In [ ]:
df_extremes_new.info()

In [ ]:
from sqlalchemy import types

In [ ]:
types.VARCHAR()

In [ ]:
%%time
df_query = pd.read_sql(sql=query, con=snowflake_ctx)

In [ ]:
pattern = channel_like\
    .replace("?", "\?")\
    .replace("/", "\/")\
    .replace("%", "(sumli|l[1,2,3])")
df_query["L"] = df_query["CHANNELID"].str.extract(pattern)

In [ ]:
df_query = df_query.apply(downcast, try_numeric=True, category=True)

In [ ]:
df_query.info()

In [ ]:
df_query.to_parquet(parquet_file, partition_cols=["BOXID", "L"])

In [ ]:
df_query

In [ ]:
cols = ["BOXID", "VALUE", "YEAR", "WEEK", "TOP", "BOTTOM", "L"]
df_read = (
    pd.read_parquet(parquet_file, columns=cols)
    .query("TOP == 1 & L == 'sumli'")
    .drop(columns=["TOP", "BOTTOM", "L"])
)
    

In [ ]:
df_read.info()

## get meta data about parquet for sake of speed

In [ ]:
import pyarrow.parquet as pq
metadata = pq.read_metadata("../data/raw/BOXID=001.622-1/L=l1/7c3663dfe57547cb8fc54c33857b6caf.parquet")

# parquet_file = pq.ParquetFile("../data/raw/BOXID=001.622-1/L=l1/7c3663dfe57547cb8fc54c33857b6caf.parquet")
# metadata = parquet_file.metadata
# metadata.row_group(0).column(6)

## see parquet engine for possibilities to read _COMMON_METEDATA
https://arrow.apache.org/docs/python/parquet.html?highlight=pyarrow%20parquet%20partition